In [1]:
import os
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache
from tqdm import tqdm, trange
from collections import Counter, defaultdict

In [2]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_2_feature.parquet'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions.csv'
train_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_train_sessions.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv'

In [3]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_train_sessions():
    return pd.read_csv(train_sessions_path)

In [4]:
def cast_dtype(df : pd.DataFrame):
    for k in df.columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [5]:
def cal_item_freq(item_counter:Counter, session_df:pd.DataFrame, test=False):
    for i in tqdm(range(session_df.shape[0])):
        sess = session_df.iloc[i]
        prev_items = eval(sess['prev_items'].replace(' ', ','))
        for item in prev_items:
            item_counter[item] += 1
        if not test:
            next_item = sess['next_item'] 
            item_counter[next_item] += 1 

# Merge test item frequency

In [6]:
merged_candidates_feature_test_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_test_2_feature.parquet'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions.csv'
train_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_train_sessions.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv'

In [7]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature_test():
    return pd.read_parquet(merged_candidates_feature_test_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_train_sessions():
    return pd.read_csv(train_sessions_path)

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [9]:
merged_candidates_feature = read_merged_candidates_feature_test()
valid_sessions_df = read_valid_sessions()
train_sessions_df = read_train_sessions()
test_sessions_df = read_test_sessions()

In [10]:
# test sessions are included in train and valid sessions
item_counter = Counter()
cal_item_freq(item_counter, train_sessions_df, test=False)
# cal_item_freq(item_counter, valid_sessions_df, test=True)
cal_item_freq(item_counter, valid_sessions_df, test=False)


100%|██████████| 361581/361581 [00:27<00:00, 13360.37it/s]


In [11]:
products, counts = zip(*item_counter.items())
item_freq_df = pd.DataFrame({'product' : products, 'product_freq' : counts})

In [30]:
# item_freq_df_g = cudf.from_pandas(item_freq_df)
# merged_candidates_feature_g = cudf.from_pandas(merged_candidates_feature_test)

In [32]:
# merged_candidates_freq_g = merged_candidates_feature_g.merge(item_freq_df_g, how='left', left_on=['product'], right_on=['product'])
# merged_candidates_freq_g = merged_candidates_freq_g.sort_values(by=['sess_id', 'product']).reset_index(drop=True)
# merged_candidates_freq_g['product_freq'] = merged_candidates_freq_g['product_freq'].fillna(0)
# cast_dtype(merged_candidates_freq_g)

In [12]:
merged_candidates_freq = merged_candidates_feature.merge(item_freq_df, how='left', left_on=['product'], right_on=['product'])
merged_candidates_freq = merged_candidates_freq.sort_values(by=['sess_id', 'product']).reset_index(drop=True)
merged_candidates_freq['product_freq'] = merged_candidates_freq['product_freq'].fillna(0)

In [ ]:
# merged_candidates_freq = merged_candidates_freq_g.to_pandas()
cast_dtype(merged_candidates_freq)
merged_candidates_freq.to_parquet(merged_candidates_feature_test_path, engine='pyarrow')

In [ ]:
# del item_freq_df_g
# del merged_candidates_feature_g
# del merged_candidates_freq_g

In [13]:
merged_candidates_freq

,sess_id,sess_locale,product,sess_avg_price,product_price,product_freq
0,0,DE,B000JG2RAG,25.195269,23.190001,24.0
1,0,DE,B000RYSOUW,25.195269,6.900000,5.0
2,0,DE,B000UGZVQM,25.195269,21.990000,4.0
3,0,DE,B000Z6JN7K,25.195269,13.170000,7.0
4,0,DE,B003CYK6FU,25.195269,11.990000,4.0
...,...,...,...,...,...,...
66438766,316970,UK,B0BJJMGPJ7,16.950001,7.990000,27.0
66438767,316970,UK,B0BJTQQWLG,16.950001,9.880000,6.0
66438768,316970,UK,B0BJV3RL4H,16.950001,22.097065,7.0
66438769,316970,UK,B0BK7SPC84,16.950001,5.960000,0.0
